# Deeper dive into Pyomo for WaterTAP Users

### [WaterTAP](https://watertap.readthedocs.io/en/stable/) is built on [IDAES](https://idaes-pse.readthedocs.io/en/stable/), which itself is built on [Pyomo](https://pyomo.readthedocs.io/en/stable/). 

Pyomo is a software package for formulating, solving, and analyzing optimization models. IDAES is a process systems modeling framework and modeling library for simulating and optimizing chemical process and energy systems that is built using Pyomo. WaterTAP builds on and extends the process simulation framework of IDAES. 

This tutorial intends to introduce some important syntax, concepts, and components from Pyomo that are used extensively in WaterTAP. Knowing how to interact with these components is important for being able to use WaterTAP. It goes step-by-step through creating a few simple Pyomo models and highlights some important syntax and methods to be aware of when interacting with these Pyomo components in WaterTAP models. Note that some IDAES components are included in this tutorial, but are not strictly necessary to create Pyomo models.

The fundamental Pyomo components used throughout WaterTAP are:
- `Block`: base container for all modeling components
- `Var`: unknown values in the model, i.e., values we are trying to optimize in our model.
- `Param`: data that must be provided in order to find optimal values for the decision variables.
- `Constraint`: equations relating model variables and parameters to each other.
- `Expression`: non-constraint relationship between modeling components.

<img src="watertap-layer-cake.png">

## Make necessary imports

The imports below represent the most commonly used Pyomo components in WaterTAP.

In [ ]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Set,
    Param,
    Constraint,
    Expression,
    check_optimal_termination,
    assert_optimal_termination,
    value,
    NonNegativeReals,
    exp,
    units as pyunits,
)
from pyomo.util.check_units import assert_units_consistent

from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.core.solvers import get_solver

## Creating Pyomo model

The `ConcreteModel` is the base object for a Pyomo and WaterTAP model. All other variables, parameters, constraints, etc. are built upon this object.
    
No arguments are necessary. Convention is to call the local object `m`.


In [ ]:
# Instantiate model as instance of a ConcreteModel object
m = ConcreteModel()

## Add a flowsheet block

Adding a `FlowsheetBlock` is not strictly necessary for pure Pyomo models, but is included here to align with
WaterTAP modeling structure and to introduce the Pyomo `Block`.

The `Block` is the basis of organization for all IDAES and WaterTAP modeling components, including unit models, property models, and the `FlowsheetBlock`.

In [ ]:
# Create a flowsheet block
m.fs = FlowsheetBlock(dynamic=False)

## Displaying the model

We can call the `.display()` method on the `FlowsheetBlock`.

This method can be used on any `Block` object and will print information 
about all constraints, variables, and objectives on that block and all sub-blocks. 

In [ ]:
# Call the display method to show variables, objectives, and constraints on the model
m.fs.display()

# You can note the difference in calling display on the flowsheet block vs. the entire model
# m.display()

## Adding variables

Throughout this tutorial, we will create a simple model for a right triangle. To represent the different legs of the right triangle, we add three variables - `A`, `B`, and `C` as `Var`

Common (and optional) keyword arguments to `Var`:
- `initialize` - the initial value for the variable
- `domain` - possible values the variable can take
- `bounds` - limits on the values the variable can take
- `units` - physical dimensions for the variable (note: more detail on units is provided later in this tutorial)
- `doc` - description of the variable

In [ ]:
m.fs.A = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable A for one leg of a right triangle",
)
m.fs.B = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable B for the other leg of a right triangle",
)
m.fs.C = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable C for the hypotenuse of a right triangle",
)

m.fs.display()

## Adding constraints

Now we add constraints relating our three variables with the Pythagorean Theorem:

### $A^2 + B^2 = C^2$

The relationship for a `Constraint` can be added in a few ways. Here, we use the `expr` keyword to directly implement the relationship. 

Note that we are using the *equality operator* (`==`) rather than the *assignment operator* (`=`).

In [ ]:
m.fs.pythagorean = Constraint(
    expr=m.fs.A**2 + m.fs.B**2 == m.fs.C**2, doc="Pythagorean theorem"
)

## Check degrees of freedom (DOF)

The degrees of freedom for the model are determined using the `degrees_of_freedom()` function from IDAES.

For running a simulation, degrees of freedom should equal zero. For running an optimization, the degrees of freedom should be greater than zero, meaning that the number of variables exceeds the number of equations by _n_ degrees of freedom.

In [ ]:
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

With two DOF, we can add another `Constraint` to further constrain the problem.

Let's say we know `A` and `B` are related with:

$ A = P \times B $

We add a `Param` to represent the ratio between `A` and `B`. The `Param` has some of the same keywords as `Var`. Here we specify `mutable=True` to indicate that we want to be able to change the parameter value after construction.

In [ ]:
m.fs.P = Param(
    initialize=3,
    mutable=True,
    units=pyunits.dimensionless,
    doc="Ratio between two legs of triangle",
)


# An alternative way to define the constraint is via a decorator.
# This syntax is commonly used in WaterTAP unit models.
@m.fs.Constraint(doc="Equation relating A and B")
def A_B_relation(b):
    return b.A == b.B * b.P


# NOTE: The above approach is equivalent to the approach below.
# m.A_B_relation = Constraint(expr=m.A == m.B * m.P, doc="Equation relating A and B")

# You can use both the display and pprint methods on many other Pyomo components, including Constraints.
m.fs.A_B_relation.pprint()
m.fs.A_B_relation.display()

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

## Fixing variables and setting parameter values

At this point, the model still has 1 DOF. We can specify the length of one of our triangle sides using the `.fix()` method on a `Var`

For `Param` components, we change the value using the `set_value()` method. The `.fix()` method will yield an error on a `Param`.

In [ ]:
m.fs.A.fix(5)
m.fs.P.set_value(23)

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

# We can just as easily unfix the variable:
m.fs.A.unfix()

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

# Solving the model

Now we create the solver by calling `get_solver()`. 

To be consistent with the WaterTAP framework, we are using the WaterTAP solver here. Alternatively, we could use the version of IPOPT from Pyomo's `SolverFactory`.

<div class="alert alert-block alert-info">
<b>Note:</b> the WaterTAP version of the solver has been tuned to better handle WaterTAP problems.
</div>

In [ ]:
# solver = SolverFactory("ipopt")
solver = get_solver()

# Fix A again
m.fs.A.fix(5)

dof = degrees_of_freedom(m)
assert dof == 0

results = solver.solve(m)

## Checking solver termination

The solve didn't give us an error, but we don't know if the solve is optimal.
Information about the solver termination is stored in the local variable `results`. We can check optimality with:
- the `assert_optimal_termination` function
- the `check_optimal_termination` function
- printing the `termination_condition` stored in `results`
- simply printing `results`

In [ ]:
# Nothing to print, but will raise an error if not optimal
assert_optimal_termination(results)

# Returns a boolean
is_optimal = check_optimal_termination(results)
print(f"Is optimal: {is_optimal}")

# Print termination condition
termination_condition = results.solver.termination_condition
print(f"Termination condition: {termination_condition}")

# Print the full results object
print(results)

## Accessing model information

You can access the value of a `Var` or `Param` by using the `value()` function.

In [ ]:
print(f"A = {value(m.fs.A)}")
print(f"B = {value(m.fs.B)}")
print(f"C = {value(m.fs.C)}")

## Working with quantities and units

Pyomo (and IDAES and WaterTAP) makes use of a package called [Pint](https://pint.readthedocs.io/en/stable/), which we have imported as `pyunits`.

`pyunits` are extremely useful for making unit conversions and ensuring models have dimensionally consistent units.

We can define any quantity with any combination of units. 

Common order-of-magnitude prefixes and their abbreviations (e.g., `mega-` and `M-`, `milli-` and `m-`) can be prepended to any unit.

Let's create 100 mg/L of some substance and a 42 MGD flow rate.

In [ ]:
C = 100 * pyunits.mg / pyunits.L
Q = 42 * pyunits.Mgal / pyunits.d
# NOTE: You can use long form unit names as well as their common abbreviations:
# C = 100 * pyunits.milligram / pyunits.liter
# Q = 42 * pyunits.megagal / pyunits.day

print(C)
print(value(C))

print(Q)
print(value(Q))

## Unit conversions

We can convert between units using `pyunits.convert()` as long as they are of the same kind.

And you can get units from any variable using `pyunits.get_units()` and passing the desired units via the `to_units` keyword.

In [ ]:
C1 = pyunits.convert(C, to_units=pyunits.lb / pyunits.ft**3)
C2 = pyunits.convert(C, to_units=pyunits.stone / pyunits.furlong**3)
C3 = pyunits.convert(C, to_units=pyunits.grain / (pyunits.acre * pyunits.foot))

Q1 = pyunits.convert(Q, to_units=pyunits.centinautical_miles**3 / pyunits.milliyear)

print(value(C1), pyunits.get_units(C1))
print(value(C2), pyunits.get_units(C2))
print(value(C3), pyunits.get_units(C3))
print(value(Q1), pyunits.get_units(Q1))

## Units carry through operations

In [ ]:
mass_flow = Q * C

print(value(mass_flow), pyunits.get_units(mass_flow))

# Or, convert to desired units directly
mass_flow = pyunits.convert(Q * C, to_units=pyunits.g / pyunits.s)

print(value(mass_flow), pyunits.get_units(mass_flow))